# Projeto Big Data para dados públicos 2024.2
### Julia Diniz
### Armando Roque Neto
### Ricardo Wurzmann


In [0]:
import pyspark.sql.functions as func

In [0]:
dados = spark.read.csv("s3://bigdata-rais/projeto/ricardow3/car_evaluation.csv")


In [0]:
dados.limit(5).display()

_c0,_c1,_c2,_c3,_c4,_c5,_c6
vhigh,vhigh,2,2,small,low,unacc
vhigh,vhigh,2,2,small,med,unacc
vhigh,vhigh,2,2,small,high,unacc
vhigh,vhigh,2,2,med,low,unacc
vhigh,vhigh,2,2,med,med,unacc


In [0]:
dados.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



## Análise Exploratória:

Quantas linhas e quantas colunas o DataFrame de receitas possui?

In [0]:
colunas = dados.columns
print(colunas)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6']


In [0]:
dados = dados.withColumnRenamed("_c0", "buying_price") \
             .withColumnRenamed("_c1", "maintenance_cost") \
             .withColumnRenamed("_c2", "number_doors") \
             .withColumnRenamed("_c3", "number_persons") \
             .withColumnRenamed("_c4", "lug_boot") \
             .withColumnRenamed("_c5", "safety") \
             .withColumnRenamed("_c6", "decision")

# Verifique se os nomes foram alterados
dados.printSchema()


root
 |-- buying_price: string (nullable = true)
 |-- maintenance_cost: string (nullable = true)
 |-- number_doors: string (nullable = true)
 |-- number_persons: string (nullable = true)
 |-- lug_boot: string (nullable = true)
 |-- safety: string (nullable = true)
 |-- decision: string (nullable = true)



In [0]:
dados.limit(5).display()

buying_price,maintenance_cost,number_doors,number_persons,lug_boot,safety,decision
vhigh,vhigh,2,2,small,low,unacc
vhigh,vhigh,2,2,small,med,unacc
vhigh,vhigh,2,2,small,high,unacc
vhigh,vhigh,2,2,med,low,unacc
vhigh,vhigh,2,2,med,med,unacc


In [0]:
print("{} linhas e {} colunas".format(dados.count(), len(dados.columns)))

1728 linhas e 7 colunas


# Iniciando a construção do modelo
Será utilizado as colunas de *[buying price]* e *[safety]* para construir um modelo de ML que consiga realizar previsões. 

Ao combinar essas duas variáveis, o modelo buscará entender como o preço de compra e a segurança estão inter-relacionados e como essa relação pode ser usada para prever resultados futuros, como a quantidade de vendas ou a satisfação do cliente.

## Transformação
 Para aplicar regressão linear em dados categóricos é preciso transformar as variáveis em valores numéricos, pois o modelo exige entradas numéricas. 

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [0]:
# Aplica o StringIndexer nas colunas desejadas
buying_price_indexer = StringIndexer(inputCol="buying_price", outputCol="buying_price_index")
safety_indexer = StringIndexer(inputCol="safety", outputCol="safety_index")

# Ajusta o indexador com todo o DataFrame para preservar o mapeamento de índices
df_indexed = buying_price_indexer.fit(dados).transform(dados)
df_indexed = safety_indexer.fit(df_indexed).transform(df_indexed)

# Divide o DataFrame em treino e teste
df_train, df_test = df_indexed.randomSplit([0.8, 0.2], seed=1234)

# Monta o vetor de features apenas com as colunas indexadas
vec_assembler = VectorAssembler(inputCols=["buying_price_index", "safety_index"], outputCol="features")


%md
## Separação da base em Treino e Teste com *ajuste*
Para verificar se o algoritmo ficou bom é preciso separar parte dos dados para teste.
Será utilizado 80% dos dados para treino e 20% para teste.

In [0]:
# Aplica a transformação nos conjuntos de treino e teste
df_train_transformed = vec_assembler.transform(df_train)
df_test_transformed = vec_assembler.transform(df_test)

# Exibe o resultado final para treino e teste
print("Conjunto de treino:")
df_train_transformed.select("buying_price_index", "safety_index", "features").show(3)

print("Conjunto de teste:")
df_test_transformed.select("buying_price_index", "safety_index", "features").show(3)

Conjunto de treino:
+------------------+------------+---------+
|buying_price_index|safety_index| features|
+------------------+------------+---------+
|               0.0|         0.0|(2,[],[])|
|               0.0|         2.0|[0.0,2.0]|
|               0.0|         0.0|(2,[],[])|
+------------------+------------+---------+
only showing top 3 rows

Conjunto de teste:
+------------------+------------+---------+
|buying_price_index|safety_index| features|
+------------------+------------+---------+
|               0.0|         1.0|[0.0,1.0]|
|               0.0|         1.0|[0.0,1.0]|
|               0.0|         0.0|(2,[],[])|
+------------------+------------+---------+
only showing top 3 rows

